## PREPROCESSING

### Install and Importing Library/Package

In [3]:
# Installing Library / Packages
%pip install tensorflow==2.15.0

Defaulting to user installation because normal site-packages is not writeable
  Using cached tensorflow-2.15.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.4 kB)
  Using cached ml_dtypes-0.2.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
  Using cached tensorboard-2.15.2-py3-none-any.whl.metadata (1.7 kB)
  Using cached keras-2.15.0-py3-none-any.whl.metadata (2.4 kB)
Using cached tensorflow-2.15.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (475.2 MB)
Using cached keras-2.15.0-py3-none-any.whl (1.7 MB)
Using cached ml_dtypes-0.2.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.0 MB)
Using cached tensorboard-2.15.2-py3-none-any.whl (5.5 MB)
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: keras
    Found existing installation: keras 3.7.0
    Uninstalling keras-3.7

In [4]:
%pip install tensorflow_decision_forests==1.8.1

Defaulting to user installation because normal site-packages is not writeable
  Using cached tensorflow_decision_forests-1.8.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.0 kB)
Using cached tensorflow_decision_forests-1.8.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (15.3 MB)
  Attempting uninstall: tensorflow_decision_forests
    Found existing installation: tensorflow_decision_forests 1.11.0
    Uninstalling tensorflow_decision_forests-1.11.0:
      Successfully uninstalled tensorflow_decision_forests-1.11.0

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install gensim

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import gensim
import numpy as np
import matplotlib.pyplot as plt
import temsorflow

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, LSTM, Embedding, Bidirectional, Input
from tensorflow.keras.utils import plot_model
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder


### Membuat Model

#### Mengimport file hasil preprocessing

In [ ]:
# Load CSV files into DataFrames
df = pd.read_csv('../datasets/output_preprocess/dataset.csv')

#### Memeriksa Dataset dan Melakukan Cleaning Tambahan

In [11]:
df.isna().sum()

text     11
label     1
dtype: int64

In [12]:
df = df.dropna()

In [14]:
df.head(15)

,text,label
0,feel really helpless heavy hearted,4.0
1,ive enjoyed able slouch relax unwind frankly n...,0.0
2,gave internship dmrg feeling distraught,4.0
3,dont know feel lost,0.0
4,kindergarten teacher thoroughly weary job take...,4.0
5,beginning feel quite disheartened,0.0
6,would think whomever would lucky enough stay s...,2.0
7,fear ever feel delicious excitement christmas ...,1.0
8,im forever taking time lie feel weird,5.0
9,still lose weight without feeling deprived,0.0


#### Menyiapkan Dataset untuk Dilatih

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42,stratify=df['label'])
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.4, random_state=42,stratify=y_test)

In [17]:
y_train

266888    3.0
69422     1.0
79034     1.0
41784     5.0
233401    0.0
         ... 
159420    1.0
24387     1.0
778       0.0
60887     3.0
61541     0.0
Name: label, Length: 259136, dtype: float64

In [20]:
#Convert the class vector (integers) to binary class matrix
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
y_val = to_categorical(y_val)

In [21]:
MAX_LENGTH = 500

# Create a tokenizer
tokenizer = Tokenizer(oov_token='UNK')
# Fit the tokenizer on the training data
tokenizer.fit_on_texts(X_train)
# Convert text to sequences of tokens
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)
X_val_seq = tokenizer.texts_to_sequences(X_val)
# Pad sequences to ensure equal length
X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LENGTH, truncating='pre')
X_test_pad = pad_sequences(X_test_seq, maxlen=MAX_LENGTH, truncating='pre')
X_val_pad = pad_sequences(X_val_seq, maxlen=MAX_LENGTH, truncating='pre')

In [22]:
# Train Word2Vec model
sentences = [text.split() for text in X_train]
w2v_model = gensim.models.Word2Vec(sentences, vector_size=200, window=5, min_count=5, workers=4)

# Create embedding matrix
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, 200))
for word, i in tokenizer.word_index.items():
    if word in w2v_model.wv:
        embedding_matrix[i] = w2v_model.wv[word]

#### Membangun arsitekteur

In [23]:
# mendefinisikan size input
vocabSize = len(tokenizer.word_index) + 1  # +1 for the padding token or OOV token
sequence_length = X_train_pad.shape[1]

# model = Sequential()
# model.add(Input(shape=(sequence_length,)))  # Input shape: (batch_size, sequence_length)
# model.add(Embedding(vocabSize, 200, weights=[embedding_matrix], input_length=sequence_length)) # Use X_train_pad to get shape
# model.add(Bidirectional(LSTM(256, dropout=0.2,recurrent_dropout=0.2, return_sequences=True)))
# model.add(Bidirectional(LSTM(128, dropout=0.2,recurrent_dropout=0.2)))
# model.add(Dense(6, activation='softmax'))

model = Sequential([
    Input(shape=(500,)),  # Adjust this shape as needed
    Embedding(vocabSize, 200, weights=[embedding_matrix]),
    Bidirectional(LSTM(256, dropout=0.2,recurrent_dropout=0.2, return_sequences=True)),
    Bidirectional(LSTM(128, dropout=0.2,recurrent_dropout=0.2)),
    Dense(6, activation='softmax')
])

adam = Adam(learning_rate=0.005)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 200)          12045000  
                                                                 
 bidirectional (Bidirection  (None, 500, 512)          935936    
 al)                                                             
                                                                 
 bidirectional_1 (Bidirecti  (None, 256)               656384    
 onal)                                                           
                                                                 
 dense (Dense)               (None, 6)                 1542      
                                                                 
Total params: 13638862 (52.03 MB)
Trainable params: 13638862 (52.03 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [25]:
class EarlyStoppingCallback(tensorflow.keras.callbacks.Callback):

    # This method is called at the end of every epoch during training
    def on_epoch_end(self, epoch, logs=None):
        # logs is a dictionary that contains metrics from the current epoch (including accuracy and val_accuracy)
        accuracy = logs.get('accuracy')  # Training accuracy
        val_accuracy = logs.get('val_accuracy')  # Validation accuracy

        # Check if training accuracy >= 0.95 and validation accuracy >= 0.80
        if accuracy >= 0.90 and val_accuracy >= 0.88:
            # Stop training
            self.model.stop_training = True
            print(f"\nReached 92% train accuracy and 88% validation accuracy at epoch {epoch+1}, stopping training!")

In [28]:
# Fit model
history = model.fit(X_train_pad,
                    y_train,
                    validation_data=(X_val_pad, y_val),
                    epochs=15,
                    batch_size=256,
                    callbacks=[EarlyStoppingCallback()]
                   )

Epoch 1/15
1013/1013 [==============================] - ETA: 0s - loss: 0.1197 - accuracy: 0.9332
Reached 92% train accuracy and 88% validation accuracy at epoch 1, stopping training!
1013/1013 [==============================] - 3225s 3s/step - loss: 0.1197 - accuracy: 0.9332 - val_loss: 0.0938 - val_accuracy: 0.9380


In [35]:
#print the overall loss and accuracy
model.evaluate(X_test_pad, y_test)

1215/1215 [==============================] - 464s 382ms/step - loss: 0.0906 - accuracy: 0.9402


[0.09057234227657318, 0.9401867985725403]

In [37]:
predicted = model.predict(X_test_pad)
y_pred = predicted.argmax(axis=-1)



1215/1215 [==============================] - 483s 396ms/step


In [ ]:
le = LabelEncoder()
y_test = np.argmax(y_test, axis=1)
print(classification_report(le.fit_transform(y_test), y_pred))

              precision    recall  f1-score   support

           0       0.98      0.97      0.98     11319
           1       0.97      0.94      0.95     13145
           2       0.81      0.91      0.86      3218
           3       0.95      0.93      0.94      5333
           4       0.92      0.89      0.91      4454
           5       0.77      0.92      0.84      1402

    accuracy                           0.94     38871
   macro avg       0.90      0.93      0.91     38871
weighted avg       0.94      0.94      0.94     38871



In [ ]:
model.save('../model/emotion-classification.h5')

/home/jupyter-220711684/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
